In [ ]:
%reload_ext autoreload
%autoreload 2
#import os
#os.environ["MKL_NUM_THREADS"] = "1"
#os.environ["NUMEXPR_NUM_THREADS"] = "1"
#os.environ["OMP_NUM_THREADS"] = "1"

from pathlib import Path
home = str(Path.home())
import sys
import numpy as np
import matplotlib.pyplot as plt
import bct
import sys
sys.path.append(home + '/workspace/networkqit')
import networkqit as nq
from networkqit.graphtheory.models.MEModels import CWTECM, UBCM, UWCM, UECM3
from networkqit.utils.visualization import plot_mle
import pandas as pd
import pickle
plt.style.use('default')

import matplotlib as mpl
mpl.rcParams.update({
    'font.family':'sans-serif',
    'font.sans-serif':['Helvetica'],
    'svg.fonttype':'none'
    })
mpl.rc('text', usetex=False)
plt.style.use('ggplot')

In [ ]:
filename = home + '//workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj'
G = np.loadtxt(filename)#[0:64, 0:64]
t = 0.2
G = bct.threshold_absolute(G,t)

tstar = G[np.nonzero(G)].min()
A = (G>0).astype(float)
k = A.sum(axis=0)
m = k.sum()
s = G.sum(axis=0)
Wtot = s.sum()
n = len(G)
pairs = n*(n-1)/2

M = UBCM(N=len(G),threshold=tstar)
sol = M.fit(G=(G>0).astype(float), method='MLE',ftol=1E-9,verbose=0)

print('Loglikelihood = ', M.loglikelihood(G,sol['x']))

nq.plot_mle(G, M.expected_adjacency(sol['x']), None)#M.expected_weighted_adjacency(sol['x']))
plt.show()

In [ ]:
batch_size=8
beta_range=np.logspace(-4,3,150)
Ws = M.sample_adjacency(theta=sol['x'],batch_size=batch_size)
S = nq.entropy(nq.graph_laplacian(G),beta_range=beta_range)
Srandom = nq.entropy(nq.graph_laplacian(M.expected_adjacency(theta=sol['x'])),beta_range=beta_range)
Srandombatch = nq.entropy(nq.graph_laplacian(Ws),beta_range=beta_range)

dklnull = nq.relative_entropy_one_component(nq.graph_laplacian(G),nq.graph_laplacian(M.expected_adjacency(theta=sol['x'])),beta_range)[0]
dklsamples = nq.relative_entropy(nq.graph_laplacian(G),nq.graph_laplacian(Ws),beta_range)[0]

dklsamplessamples=[]
for i in range(batch_size):
    dklsamplessamples.append(nq.relative_entropy_one_component(nq.graph_laplacian(Ws)[i,...],nq.graph_laplacian(Ws),beta_range)[0])
dklsamplessamples=np.asarray(dklsamplessamples).T

In [ ]:
fig, ax = plt.subplots(figsize=(24,7),ncols=2)
ax[0].semilogx(beta_range,S,label='data')
ax[0].semilogx(beta_range,Srandom,label='null',linewidth=2)
ax[0].semilogx(beta_range,Srandombatch,label='null batch',color='k',alpha=0.2)
ax[0].fill_between(beta_range,Srandombatch.mean(1)-Srandombatch.std(1),Srandombatch.mean(1)+Srandombatch.std(1),color='k',alpha=0.1)
ax[0].legend(['$S(\\rho)$','$S(\\sigma(E[L]))$','$S_i(\\sigma(L_i))$'])
ax[0].grid(True)
ax[0].set_title('Spectral entropy')

ax[1].semilogx(beta_range,dklnull)
ax[1].semilogx(beta_range,dklsamples)
ax[1].semilogx(beta_range,dklsamplessamples,label='samples vs samples',color='k',alpha=0.1)
ax[1].legend(['$S(\\rho \| \\sigma(E[L]))$','$E[S(\\rho \\| \\sigma)]$','$S(\\sigma_i \| \sigma_j)$'])
ax[1].set_title('Relative entropy')
ax[1].grid(True)
#ax[1].set_ylim([0,4])
plt.savefig('entropies_dkl_638_thresh_02_ubcm.pdf',bbox_inches='tight')
plt.show()

In [ ]:
nq.plot_mle(G, np.squeeze(sample_adjacency(M,sol['x']))[1], np.squeeze(sample_adjacency(M,sol['x'])[0]))

In [ ]:
nq.plot_mle(G, (sample_adjacency(M,sol['x'],batch_size=100)[1].mean(0)).astype(float),
                np.mean(sample_adjacency(M,sol['x'],batch_size=100)[0],0))

In [ ]:
_=plt.hist(G[np.nonzero(G)],200,label='data',alpha=0.6,density=True)
_=plt.hist(np.squeeze(sample_adjacency(M,sol['x'])).flatten(),200,label='null',alpha=0.6,density=True)
plt.legend()

In [ ]:
ws=np.squeeze(sample_adjacency(M,sol['x'])[0])
ws=ws[np.nonzero(G)]
plt.plot(G[np.nonzero(G)],ws,'.')

In [ ]:
G = np.random.exponential(scale=0.01,size=(256,256))

In [ ]:
ws = M.sample_adjacency(theta=sol['x'])
_=plt.hist(W[np.nonzero(W)],label='data',bins=50,alpha=0.5,density=True)
_=plt.hist(ws.flat[ws.flat>0]+25,label='null',bins=50,alpha=0.5,density=True)
plt.legend()
plt.show()